# Shark Attacks

Objetivos:

Identificar se fatalidades estão relacionadas a alguma espécie específica;

Identificar se ataques ocorrem em uma época específica e porque.

In [12]:
import os
import pandas as pd
import numpy as np

In [30]:
shark_data = pd.read_csv("attacks.csv")

## Dados originais

In [14]:
shark_data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson�McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [15]:
shark_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

## Retirada de colunas que não serão utilizadas e limpeza das restantes

In [22]:
shark_data.drop('Name', axis=1)
cshark_data = shark_data.drop(['Name', 'original order', 'Investigator or Source', 'pdf', 'href formula', 'Case Number.1','Case Number.2', 'Unnamed: 22', 'Unnamed: 23'], axis=1)

In [23]:
shark_columns = list(cshark_data.columns)
shark_clean_col = [col.lower().strip().replace(' ','_') for col in shark_columns]

In [24]:
cshark_data.columns = shark_clean_col
cshark_data.head(50)

,case_number,date,year,type,country,area,location,activity,sex,age,injury,fatal_(y/n),time,species,href
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,http://sharkattackfile.net/spreadsheets/pdf_di...
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",http://sharkattackfile.net/spreadsheets/pdf_di...
5,2018.06.03.b,03-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,M,NaN,"No injury, board bitten",N,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...
6,2018.06.03.a,03-Jun-2018,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,M,18,FATAL,Y,Late afternoon,Tiger shark,http://sharkattackfile.net/spreadsheets/pdf_di...
7,2018.05.27,27-May-2018,2018.0,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,M,52,Minor injury to foot. PROVOKED INCIDENT,N,NaN,"Lemon shark, 3'",http://sharkattackfile.net/spreadsheets/pdf_di...
8,2018.05.26.b,26-May-2018,2018.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,M,15,Lower left leg bitten,N,17h00,"Bull shark, 6'",http://sharkattackfile.net/spreadsheets/pdf_di...
9,2018.05.26.a,26-May-2018,2018.0,Unprovoked,USA,Florida,"Daytona Beach, Volusia County",Standing,M,12,Minor injury to foot,N,14h00,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...


In [193]:
cshark_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   case_number  8702 non-null   object 
 1   date         6302 non-null   object 
 2   year         6300 non-null   float64
 3   type         6298 non-null   object 
 4   country      6252 non-null   object 
 5   area         5847 non-null   object 
 6   location     5762 non-null   object 
 7   activity     5758 non-null   object 
 8   sex          5737 non-null   object 
 9   age          3471 non-null   object 
 10  injury       6274 non-null   object 
 11  fatal_(y/n)  5763 non-null   object 
 12  time         2948 non-null   object 
 13  species      3464 non-null   object 
 14  href         6302 non-null   object 
dtypes: float64(1), object(14)
memory usage: 2.9+ MB


## Primeira análise dos dados

In [421]:
cshark_data.describe()

,year
count,6300.000000
mean,1927.272381
std,281.116308
min,0.000000
25%,1942.000000
50%,1977.000000
75%,2005.000000
max,2018.000000


In [ ]:
#A a maior parte nos estados unidos, surfando, homens, 17 anos, perto de 2015,
#verificar injury pois não está coerente, a tarde, tubarões brancos.

## Identificação dos dados unicos de cada coluna e verificar o que precisa ser limpo

In [197]:
cshark_data['type'].unique()

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', nan, 'Boat', 'Boatomg'], dtype=object)

In [66]:
cshark_data['type'].value_counts()

Unprovoked      4595
Provoked         574
Invalid          547
Sea Disaster     239
Boating          203
Boat             137
Questionable       2
Boatomg            1
Name: type, dtype: int64

In [196]:
cshark_data['country'].unique()

array(['USA', 'AUSTRALIA', 'MEXICO', 'BRAZIL', 'ENGLAND', 'SOUTH AFRICA',
       'THAILAND', 'COSTA RICA', 'MALDIVES', 'BAHAMAS', 'NEW CALEDONIA',
       'ECUADOR', 'MALAYSIA', 'LIBYA', nan, 'CUBA', 'MAURITIUS',
       'NEW ZEALAND', 'SPAIN', 'SAMOA', 'SOLOMON ISLANDS', 'JAPAN',
       'EGYPT', 'ST HELENA, British overseas territory', 'COMOROS',
       'REUNION', 'FRENCH POLYNESIA', 'UNITED KINGDOM',
       'UNITED ARAB EMIRATES', 'PHILIPPINES', 'INDONESIA', 'CHINA',
       'COLUMBIA', 'CAPE VERDE', 'Fiji', 'DOMINICAN REPUBLIC',
       'CAYMAN ISLANDS', 'ARUBA', 'MOZAMBIQUE', 'FIJI', 'PUERTO RICO',
       'ITALY', 'ATLANTIC OCEAN', 'GREECE', 'ST. MARTIN', 'FRANCE',
       'PAPUA NEW GUINEA', 'TRINIDAD & TOBAGO', 'KIRIBATI', 'ISRAEL',
       'DIEGO GARCIA', 'TAIWAN', 'JAMAICA', 'PALESTINIAN TERRITORIES',
       'GUAM', 'SEYCHELLES', 'BELIZE', 'NIGERIA', 'TONGA', 'SCOTLAND',
       'CANADA', 'CROATIA', 'SAUDI ARABIA', 'CHILE', 'ANTIGUA', 'KENYA',
       'RUSSIA', 'TURKS & CAICOS', 'UNITE

In [198]:
cshark_data['country'].value_counts()

USA                             2229
AUSTRALIA                       1338
SOUTH AFRICA                     579
PAPUA NEW GUINEA                 134
NEW ZEALAND                      128
                                ... 
GREENLAND                          1
EQUATORIAL GUINEA / CAMEROON       1
RED SEA?                           1
MID-PACIFC OCEAN                   1
WESTERN SAMOA                      1
Name: country, Length: 212, dtype: int64

In [57]:
cshark_data['age'].unique()

array(['57', '11', '48', nan, '18', '52', '15', '12', '32', '10', '21',
       '34', '30', '60', '33', '29', '54', '41', '37', '56', '19', '25',
       '69', '38', '55', '35', '46', '45', '14', '40s', '28', '20', '24',
       '26', '49', '22', '7', '31', '17', '40', '13', '42', '3', '8',
       '50', '16', '82', '73', '20s', '68', '51', '39', '58', 'Teen',
       '47', '61', '65', '36', '66', '43', '60s', '9', '72', '59', '6',
       '27', '64', '23', '71', '44', '62', '63', '70', '18 months', '53',
       '30s', '50s', 'teen', '77', '74', '28 & 26', '5', '86', '18 or 20',
       '12 or 13', '46 & 34', '28, 23 & 30', 'Teens', '36 & 26',
       '8 or 10', '84', '� ', ' ', '30 or 36', '6�', '21 & ?', '75',
       '33 or 37', 'mid-30s', '23 & 20', ' 30', '7      &    31', ' 28',
       '20?', "60's", '32 & 30', '16 to 18', '87', '67', 'Elderly',
       'mid-20s', 'Ca. 33', '74 ', '45 ', '21 or 26', '20 ', '>50',
       '18 to 22', 'adult', '9 & 12', '? & 19', '9 months', '25 to 35',
     

In [58]:
cshark_data['age'].value_counts()

17         154
18         150
19         142
20         141
15         139
          ... 
�            1
23 & 26      1
Elderly      1
adult        1
67           1
Name: age, Length: 157, dtype: int64

In [59]:
cshark_data['activity'].unique()

array(['Paddling', 'Standing', 'Surfing', ...,
       'Crew swimming alongside their anchored ship',
       '4 men were bathing', 'Wreck of  large double sailing canoe'],
      dtype=object)

In [60]:
cshark_data['activity'].value_counts()

Surfing                                             971
Swimming                                            869
Fishing                                             431
Spearfishing                                        333
Bathing                                             162
                                                   ... 
Swimming between  anchored  pearling luggers          1
Swimming to shore from capsized sailboat              1
U-177 torpedoed & sank the troopship Nova Scotia      1
Swimming 10 m from shore                              1
Swimming after his canoe capsized                     1
Name: activity, Length: 1532, dtype: int64

In [61]:
cshark_data['species'].unique()

array(['White shark', nan, '2 m shark', ..., "12' tiger shark",
       'Blue pointers',
       'Said to involve a grey nurse shark that leapt out of the water and  seized the boy but species identification is questionable'],
      dtype=object)

In [199]:
cshark_data['species'].value_counts()

White shark                                           163
Shark involvement prior to death was not confirmed    105
Invalid                                               102
Shark involvement not confirmed                        88
Tiger shark                                            73
                                                     ... 
5' to 8' shark                                          1
3.3 m [10'9"] shark                                     1
Blacktip reef shark pup                                 1
White shark, 12' to 15'                                 1
Nurse shark, 1.1 m [3.5']                               1
Name: species, Length: 1549, dtype: int64

In [63]:
cshark_data['year'].unique()

array([2018.0, 2017.0, nan, 2016.0, 2015.0, 2014.0, 2013.0, 2012.0,
       2011.0, 2010.0, 2009.0, 2008.0, 2007.0, 2006.0, 2005.0, 2004.0,
       2003.0, 2002.0, 2001.0, 2000.0, 1999.0, 1998.0, 1997.0, 1996.0,
       1995.0, 1984.0, 1994.0, 1993.0, 1992.0, 1991.0, 1990.0, 1989.0,
       1969.0, 1988.0, 1987.0, 1986.0, 1985.0, 1983.0, 1982.0, 1981.0,
       1980.0, 1979.0, 1978.0, 1977.0, 1976.0, 1975.0, 1974.0, 1973.0,
       1972.0, 1971.0, 1970.0, 1968.0, 1967.0, 1966.0, 1965.0, 1964.0,
       1963.0, 1962.0, 1961.0, 1960.0, 1959.0, 1958.0, 1957.0, 1956.0,
       1955.0, 1954.0, 1953.0, 1952.0, 1951.0, 1950.0, 1949.0, 1948.0,
       1848.0, 1947.0, 1946.0, 1945.0, 1944.0, 1943.0, 1942.0, 1941.0,
       1940.0, 1939.0, 1938.0, 1937.0, 1936.0, 1935.0, 1934.0, 1933.0,
       1932.0, 1931.0, 1930.0, 1929.0, 1928.0, 1927.0, 1926.0, 1925.0,
       1924.0, 1923.0, 1922.0, 1921.0, 1920.0, 1919.0, 1918.0, 1917.0,
       1916.0, 1915.0, 1914.0, 1913.0, 1912.0, 1911.0, 1910.0, 1909.0,
       19

In [65]:
cshark_data['year'].value_counts()

2015.0    143
2017.0    136
2016.0    130
2011.0    128
2014.0    127
         ... 
1807.0      1
1805.0      1
1804.0      1
1802.0      1
1791.0      1
Name: year, Length: 249, dtype: int64

# Limpeza da tabela

Há muitas linhas sem valor nenhum. Retirar as linhas com muitos NaN

In [28]:
cshark_data.tail(100)

,case_number,date,year,type,country,area,location,activity,sex,age,injury,fatal_(y/n),time,species,href
25623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25624,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25627,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
cshark = cshark_data.dropna(thresh = 0.1*len(cshark_data.columns), axis=0)

## Limpeza das datas

Limpar as datas para aplicação de .to_datetime 

In [156]:
import re

In [200]:
 re.findall('\d{2}-\D{3}-\d+', "Before 13-jun-1998") #Teste de Regex

['13-jun-1998']

In [158]:
def change_date(name):
    date1 = re.findall('\d{2}-\D{3}-\d+', name)
    if len(date1)>0:
        return date1[0]

In [159]:
cshark['date'] = cshark['date'].apply(change_date)

<ipython-input-159-803a69df7c4a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cshark['date'] = cshark['date'].apply(change_date)


In [169]:
cshark['date'] = pd.to_datetime(cshark['date'])

In [171]:
cshark.info() #Verificação do formato da coluna Date

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6300 entries, 0 to 6301
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   case_number  6299 non-null   object        
 1   date         5348 non-null   datetime64[ns]
 2   year         6298 non-null   float64       
 3   type         6296 non-null   object        
 4   country      6250 non-null   object        
 5   area         5846 non-null   object        
 6   location     5761 non-null   object        
 7   activity     5756 non-null   object        
 8   sex          5735 non-null   object        
 9   age          3470 non-null   object        
 10  injury       6272 non-null   object        
 11  fatal_(y/n)  5761 non-null   object        
 12  time         2947 non-null   object        
 13  species      3463 non-null   object        
 14  href         6300 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(13)
memory usa

In [166]:
cshark['date'][4600:4650]


4600    19-Apr-1946
4601           None
4602    20-Mar-1946
4603    16-Feb-1946
4604    10-Feb-1946
4605    10-Feb-1946
4606    24-Jan-1946
4607    24-Jan-1946
4608    17-Jan-1946
4609    07-Jan-1946
4610    05-Jan-1946
4611    01-Jan-1946
4612           None
4613           None
4614           None
4615    23-Sep-1945
4616    19-Sep-1945
4617    16-Sep-1945
4618    06-Sep-1945
4619    19-Aug-1945
4620    06-Aug-1945
4621    30-Jul-1945
4622           None
4623    15-Jun-1945
4624    08-May-1945
4625    06-Mar-1945
4626    05-Feb-1945
4627           None
4628           None
4629           None
4630           None
4631           None
4632    02-Dec-1944
4633    08-Nov-1944
4634    01-Nov-1944
4635    26-Oct-1944
4636    26-Oct-1944
4637    25-Oct-1944
4638    25-Oct-1944
4639    24-Oct-1944
4640    23-Oct-1944
4641    03-Sep-1944
4642           None
4643    20-Aug-1944
4644     22-Jul-144
4645    23-Jun-1944
4646    31-May-1944
4647    24-May-1944
4648    04-May-1944
4649           None


In [167]:
cshark=cshark.drop(index=[4644])

## Limpeza das espécies

In [201]:
cshark['species'].value_counts().head(50)

White shark                                           163
Shark involvement prior to death was not confirmed    105
Invalid                                               102
Shark involvement not confirmed                        88
Tiger shark                                            73
Shark involvement prior to death unconfirmed           68
Bull shark                                             52
4' shark                                               40
6' shark                                               40
1.8 m [6'] shark                                       35
Questionable incident                                  35
Questionable                                           34
1.5 m [5'] shark                                       32
1.2 m [4'] shark                                       27
5' shark                                               26
3' shark                                               26
2 m shark                                              25
4' to 5' shark

In [204]:
cshark[cshark['species']=="Invalid"].head(30)

,case_number,date,year,type,country,area,location,activity,sex,age,injury,fatal_(y/n),time,species,href
226,2016.09.00,NaT,2016.0,Invalid,MEXICO,NaN,Guadalupe Island,Cage Diving,NaN,NaN,"No injury to divers, white shark breached cage",NaN,NaN,Invalid,http://sharkattackfile.net/spreadsheets/pdf_di...
578,2014.03.13,2014-03-13,2014.0,Invalid,CAYMAN ISLANDS,NaN,NaN,Scuba diving / culling lionfish,M,NaN,"Caribbean reef shark buzzed him. No injury, no...",NaN,NaN,Invalid,http://sharkattackfile.net/spreadsheets/pdf_di...
676,2013.05.08.b,2013-05-08,2013.0,Invalid,USA,California,"Tourmaline Surf Park, San Diego County",Surfing,M,42,Shark bites were post-mortem,NaN,NaN,Invalid,http://sharkattackfile.net/spreadsheets/pdf_di...
926,2011.05.07.R,2011-05-07,2011.0,Invalid,UNITED ARAB EMIRATES (UAE),Umm al Qaywayan Province,Khor Fakkan,Fishing,M,43,Erroneously reported on several internet sites...,NaN,NaN,Invalid,http://sharkattackfile.net/spreadsheets/pdf_di...
1044,2010.02.04,2010-02-04,2010.0,Invalid,GUAM,Merizo,Achang Reef,Spearfishing (free diving),M,31,Shark bites were post-mortem,NaN,11h00,Invalid,http://sharkattackfile.net/spreadsheets/pdf_di...
1093,2009.09.02,2009-09-02,2009.0,Invalid,NEVIS,NaN,Castle Beach,Swimming,M,NaN,Death was due to drowning. Two days later his ...,NaN,NaN,Invalid,http://sharkattackfile.net/spreadsheets/pdf_di...
1167,2009.01.23,2009-01-23,2009.0,Invalid,BRAZIL,Maranh�o,Olho d'�gua,Swimming,M,17,"Drowned, body scavenged by shark",NaN,NaN,Invalid,http://sharkattackfile.net/spreadsheets/pdf_di...
1301,2007.12.19,2007-12-19,2007.0,Invalid,BRITISH VIRGIN ISLANDS,Green Bay,NaN,Scuba diving,M,53,Shark bites were post-mortem,NaN,NaN,Invalid,http://sharkattackfile.net/spreadsheets/pdf_di...
1314,2007.11.00,NaT,2007.0,Invalid,MEXICO,Baja California,Guadalupe Island,Shark diving,M,NaN,White shark breached cage. No injury to occupants,NaN,NaN,Invalid,http://sharkattackfile.net/spreadsheets/pdf_di...
1427,2006.09.30,2006-09-30,2006.0,Invalid,SOUTH AFRICA,Western Cape Province,Miller's Point,Spearfishing,M,36,No injury; 4m white shark made a threat display,NaN,NaN,Invalid,http://sharkattackfile.net/spreadsheets/pdf_di...


In [250]:
re.sub('.* ?[Ww]hite [Ss]hark.*', 'WhiteShark','a huge white shark said hello and invited the man to dinner.')

'WhiteShark'

In [273]:
re.sub('.* [Ss]hark.*', 'puppy','WhiteShark ')

'WhiteShark '

In [271]:
re.sub('.* ?[Ss]hark.*', 'puppy','White Shark ')

'puppy'

In [488]:
shark_dict1 = {'.* ?[Qq]uestionable ?.*': 'Invalid',
               '.* ?[Nn]ot [Cc]onfirmed ?.*': 'Invalid',
               '.* ?[Ww]hite [Ss]hark ?.*': 'WhiteShark', 
               '.* ?[Bb]ull [Ss]hark ?.*': 'BullShark', 
               '.* ?[Tt]iger [Ss]hark ?.*': 'TigerShark', 
               '.* ?[Nn]urse [Ss]hark ?.*': 'NurseShark',
               '.* ?[Nn]o [Ss]hark ?.*': 'Invalid',
               '.* ?[Hh]ammerhead [Ss]hark ?.*': 'HammerheadShark',
               '.* ?[Bb]ronze ?[Ww]haler [Ss]hark ?.*': 'BronzewhalerShark',
               '.* ?[Ww]haler [Ss]hark ?.*': 'WhalerShark',
               '.* ?[Ww]obbegong[ s][,]?  [Ss]hark ?.*': 'WobbegongShark',
               '.* ?[Ww]obbegong[,]? ?.*': 'WobbegongShark',
               '.* ?[Bb]lacktip [Ss]hark ?.*': 'BlacktipShark',
               '.* ?[Ll]emon [Ss]hark ?.*': 'LemonShark',
               '.* ?[Bb]lue [Ss]hark ?.*': 'BlueShark',
               '.* ?[Ww]hitetip [Ss]hark ?.*': 'WhitetipShark',
               '.* ?[Gg]r[ae]y [Rr]eef [Ss]hark ?.*': 'GreyreefShark',
               '.* ?[Bb]lue [Pp]ointers ?.*': 'BluePointers'}

In [489]:
def change_spec1(name,shark_dict1,key):
    if isinstance(name,str):
        return re.sub(key, shark_dict1[key], name)
    else:
        return None

In [483]:
cshark['species'] = cshark['species'].apply(change_spec1, shark_dict1=shark_dict1, key='.* [Ww]hite [Ss]hark .*')

KeyError: '.* [Ww]hite [Ss]hark .*'

In [490]:
for key in shark_dict1.keys():
    cshark['species'] = cshark['species'].apply(change_spec1, shark_dict1=shark_dict1, key=key)

In [334]:
shark_dict2 = {'.* [Ss]hark ?.*': 'UnknownShark',
               '.* [Dd]rowining ?.*': 'Invalid',
               '.* [Uu]nconfirmed ?.*': 'Invalid',
               '.* [Tt]o [Bb]e [Cc]onfirmed ?.*': 'Invalid',
               '.* ?\d+ m .*': 'UnknownShark',
               ".* ?\d+ cm .*": 'UnknownShark',
               '.* ?[Ii]nvalid ?.*': 'Invalid'}

In [335]:
def change_unknown(name,shark_dict2,key):
    if isinstance(name,str):
        return re.sub(key, shark_dict2[key], name)
    else:
        return None

In [485]:
for key in shark_dict2.keys():
    cshark['species'] = cshark['species'].apply(change_unknown, shark_dict2=shark_dict2, key=key)

In [486]:
cshark['species'].value_counts()

UnknownShark                           1409
WhiteShark                              634
Invalid                                 504
TigerShark                              281
BullShark                               180
                                       ... 
"sandshark"                               1
Tooth fragments recovered from hull       1
15'                                       1
BluePointers                              1
"Shark caught later"                      1
Name: species, Length: 61, dtype: int64

In [347]:
cshark

,case_number,date,year,type,country,area,location,activity,sex,age,injury,fatal_(y/n),time,species,href
0,2018.06.25,2018-06-25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,WhiteShark,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2018.06.18,2018-06-18,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,F,11,Minor injury to left thigh,N,14h00 -15h00,None,http://sharkattackfile.net/spreadsheets/pdf_di...
2,2018.06.09,2018-06-09,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,M,48,Injury to left lower leg from surfboard skeg,N,07h45,None,http://sharkattackfile.net/spreadsheets/pdf_di...
3,2018.06.08,2018-06-08,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,M,NaN,Minor injury to lower leg,N,NaN,UnknownShark,http://sharkattackfile.net/spreadsheets/pdf_di...
4,2018.06.04,2018-06-04,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,TigerShark,http://sharkattackfile.net/spreadsheets/pdf_di...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,NaT,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,M,NaN,FATAL,Y,NaN,None,http://sharkattackfile.net/spreadsheets/pdf_di...
6298,ND.0004,NaT,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,M,NaN,FATAL,Y,NaN,None,http://sharkattackfile.net/spreadsheets/pdf_di...
6299,ND.0003,NaT,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,M,NaN,FATAL,Y,NaN,None,http://sharkattackfile.net/spreadsheets/pdf_di...
6300,ND.0002,NaT,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8�N, 79�W",NaN,M,NaN,FATAL,Y,NaN,None,http://sharkattackfile.net/spreadsheets/pdf_di...


In [348]:
cshark = cshark.drop(['sex', 'age', 'case_number', 'href'], axis=1)

In [349]:
cshark

,date,year,type,country,area,location,activity,injury,fatal_(y/n),time,species
0,2018-06-25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,"No injury to occupant, outrigger canoe and pad...",N,18h00,WhiteShark
1,2018-06-18,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Minor injury to left thigh,N,14h00 -15h00,None
2,2018-06-09,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,Injury to left lower leg from surfboard skeg,N,07h45,None
3,2018-06-08,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,Minor injury to lower leg,N,NaN,UnknownShark
4,2018-06-04,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,TigerShark
...,...,...,...,...,...,...,...,...,...,...,...
6297,NaT,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,FATAL,Y,NaN,None
6298,NaT,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,FATAL,Y,NaN,None
6299,NaT,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,FATAL,Y,NaN,None
6300,NaT,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8�N, 79�W",NaN,FATAL,Y,NaN,None


## Limpeza da coluna fatal

A coluna injury e a coluna fatal não apresentam dados coerentes entre elas. Verificar.

In [431]:
shark_dict3 = {' ?[Nn] ?': 'N',
               ' ?[Yy] ?': 'Y',
               ' ?[Mm] ?': 'N',
               ' ?UNKNOWN ?': 'UNKNOWN',
               '\d+': 'UNKNOWN'}

In [432]:
def change_inj(name,shark_dict3,key):
    if isinstance(name,str):
        return re.sub(key, shark_dict3[key], name)
    else:
        return None

In [433]:
for key in shark_dict3.keys():
    cshark['fatal_(y/n)'] = cshark['fatal_(y/n)'].apply(change_inj, shark_dict3=shark_dict3, key=key)

In [434]:
cshark['fatal_(y/n)'].value_counts()

N          4302
Y          1387
UNKNOWN      72
Name: fatal_(y/n), dtype: int64

## Limpeza Locais

In [473]:
cshark['area'].value_counts()

Florida                              1037
New South Wales                       486
Queensland                            311
Hawaii                                298
California                            290
                                     ... 
Cavite Province, Luzon                  1
Island of Volos                         1
Makora-Ulawa Province                   1
Phang nga Province                      1
Between New Ireland & New Britain       1
Name: area, Length: 825, dtype: int64

In [446]:
cshark['year'] = cshark['year'].astype(object)

In [496]:
pd.options.display.max_rows=100

In [499]:
cshark.groupby(['fatal_(y/n)', 'species'], as_index=False).count().sort_values(['fatal_(y/n)','year'], ascending = False)

,fatal_(y/n),species,date,year,type,country,area,location,activity,injury,time
69,Y,WhiteShark,136,143,143,143,140,141,139,143,113
67,Y,UnknownShark,114,137,137,135,126,130,128,137,63
65,Y,TigerShark,57,69,69,69,64,64,66,69,41
58,Y,BullShark,34,40,40,40,36,37,35,40,24
61,Y,Invalid,6,9,9,8,8,9,9,9,3
56,Y,BlueShark,7,7,7,6,5,5,5,7,6
70,Y,WhitetipShark,5,5,5,5,5,5,5,5,3
57,Y,BronzewhalerShark,4,4,4,4,4,4,4,4,4
54,Y,Blue pointer,0,2,2,2,0,0,2,2,0
59,Y,HammerheadShark,1,2,2,2,2,1,2,2,0


In [501]:
cshark['month'] = cshark['date'].dt.month

In [502]:
cshark

,date,year,type,country,area,location,activity,injury,fatal_(y/n),time,species,month
0,2018-06-25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,No injury,N,18h00,WhiteShark,6.0
1,2018-06-18,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Minor injury to left thigh,N,14h00 -15h00,None,6.0
2,2018-06-09,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,Injury to left lower leg from surfboard skeg,N,07h45,None,6.0
3,2018-06-08,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,Minor injury to lower leg,N,NaN,UnknownShark,6.0
4,2018-06-04,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,TigerShark,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6297,NaT,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,FATAL,Y,NaN,None,NaN
6298,NaT,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,FATAL,Y,NaN,None,NaN
6299,NaT,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,FATAL,Y,NaN,None,NaN
6300,NaT,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8�N, 79�W",NaN,FATAL,Y,NaN,None,NaN
